<p style="color:#4169E1; font-family:'Roboto'; font-size:48px; text-align: center;"> 
Process WLS data
</p>

<span style='font-size:32px; color:magenta;'>This is prototype code for process_wls_data.py script.</span>

**Table of contents**<a id='toc0_'></a>    
- 1. [Change diary:](#toc1_)    
- 2. [Reading pickled event catalogue](#toc2_)    
- 3. [initialize Pyreco](#toc3_)    
  - 3.1. [Event information](#toc3_1_)    
- 4. [Analysis](#toc4_)    
  - 4.1. [data selection](#toc4_1_)    
  - 4.2. [Plotting data](#toc4_2_)    
  - 4.3. [waveform sum](#toc4_3_)    
    - 4.3.1. [filtered waveform sum](#toc4_3_1_)    
    - 4.3.2. [pretrigger sum](#toc4_3_2_)    
    - 4.3.3. [negative wf sum](#toc4_3_3_)    
    - 4.3.4. [Repetitive pattern seen in ch-2 data](#toc4_3_4_)    
      - 4.3.4.1. [Fourier Transform](#toc4_3_4_1_)    
  - 4.4. [simultaneity of pulses](#toc4_4_)    
    - 4.4.1. [Create Pulse difference Distribution](#toc4_4_1_)    
  - 4.5. [Center of Mass for waveforms](#toc4_5_)    
  - 4.6. [full waveform sum](#toc4_6_)    
  - 4.7. [Cuts](#toc4_7_)    
- 5. [Exploration](#toc5_)    
- 6. [time constant](#toc6_)    
- 7. [Memory diagnostics](#toc7_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

<p>
<b> Steps to follow: </b>

<ol>
<li> Event selection cuts </li>
<li> Histogram event charge</li>
<li> SPE calibration </li>
<li> cut efficiency: waveform integration post cut / pre cut bin by bin</li>
<li> Light Yield calculation</li>
<!-- <li> </li> -->
</ol>
</p>


<!-- # MIDAS events reconstruction -->

Load packages, setup libraries

In [ ]:
from time import perf_counter
t0 = perf_counter()

In [ ]:
import sys
import numpy as np
from scipy.optimize import curve_fit
# from scipy.stats import chisquare
# from scipy.integrate import simpson
# import matplotlib
import matplotlib.pyplot as plt
# from matplotlib.ticker import AutoMinorLocator
# from matplotlib.ticker import FormatStrFormatter
# from termcolor import colored
# import latexify
import pandas as pd
# from tqdm.notebook import tqdm
import pickle
from os import path
from scipy.signal import find_peaks

In [ ]:
np.set_printoptions(formatter={'float': lambda x: f"{x:10.4g}"})

In [ ]:
%matplotlib ipympl

# 1. <a id='toc1_'></a>[Change diary:](#toc0_)
1. Marcin suggested to use these settings in pyreco for some runs:
`n32samples = 9168-1`
2. new pyreco branch: zle_4ksamp_length
3. Marcin suggested 3 WLS specific cuts
4. LY calculation w/o SPE calibration
5. LY calculation w/ SPE calibration

# 2. <a id='toc2_'></a>[Reading pickled event catalogue](#toc0_)

In [ ]:
# event_catalogue_file = '../temp_folder/copy_event_catalogue_run0061.pkl'
# filename = 'event_catalogue_run0061.pkl'
# event_catalogue_file = '../data/event_catalogue_run00061_3ch.pkl'
# filename = 'event_catalogue_run00108_first_half.pkl'
# filename = 'event_catalogue_run00126_part.pkl'
# filename = 'event_catalogue_run00152_00.pkl'
# filename = 'event_catalogue_run00162_00.pkl'
# filename = 'event_catalogue_run00126.pkl'
filename = 'event_catalogue_run00126_part.pkl'

data_dir = '/work/chuck/sarthak/argset/event_catalogues'
event_catalogue_file = path.join(data_dir, filename)
event_catalogue = pd.read_pickle(event_catalogue_file)
wfs = event_catalogue['wf']
del event_catalogue

In [ ]:
# Plotting events
plt.figure()
# for event_id in range(9900, 9905):
for event_id in range(5000, 5001):
    for ch in range(3):
    # for ch in [2]:
        plt.plot(wfs[event_id][ch])

In [ ]:
# --> 1&2 waveforms shouldn't have maximum shifted by more than ~ 20 ns

# 3. <a id='toc3_'></a>[initialize Pyreco](#toc0_)

Select here midas data, output filename and configuration file

In [ ]:
from pyreco.manager.manager import Manager

# # filename = '/work/sarthak/argset/data/2024_Mar_27/midas/run00061.mid.lz4'
filename = '/work/sarthak/argset/data/run00152_00.mid.lz4'
# filename = '/work/sarthak/argset/data/run00152.mid.lz4'
outfile = 'jupyR00152'
confile = 'argset.ini'
tmin,tmax = 0, 4000
# # tmin,tmax = 0, 1750
cmdline_args = f'--config {confile} -o {outfile} -i {filename}'
m = Manager( midas=True, cmdline_args=cmdline_args)
# m = Manager( midas=True) #, cmdline_args=cmdline_args)
# baseline_samples = m.config('reco', 'bl_to', 'int')
from pyreco.reco.filtering import WFFilter
mfilter = WFFilter(m.config)

In [ ]:
%matplotlib ipympl

In [ ]:
def perform_arma(og_wf):
    flt = np.reshape(mfilter.numba_fast_filter(og_wf), newshape=og_wf.shape)
    # mas = m.algos.running_mean(flt, gate=60)
    # return flt - mas
    return flt

In [ ]:
event_id = 49959

fig_11, ax_11 = plt.subplots(2,1)
for ch_id in range(3):
    ax_11[0].plot(wfs[event_id][ch_id], label = f'og {ch_id}')
    ax_11[0].legend()
    ax_11[1].plot(perform_arma(wfs[event_id])[ch_id], label = f'flt {ch_id}')
    ax_11[1].legend()
plt.show()

In [ ]:
# fig_11, ax_11 = plt.subplots(2,1, figsize=(15, 12))
# for ch_id in range(3):
#     ax_11[0].scatter(np.arange(50_000), np.array(wf_sum_dict[ch_id]), marker='o', color='green', label = f'og {ch_id}')
#     ax_11[0].scatter(negative_sums, np.array(wf_sum_dict[ch_id])[negative_sums], marker='+', color='red', label = f'og {ch_id}')
#     # ax_11[0].legend()
#     ax_11[0].set_xlim((0, 5000))
#     ax_11[0].set_ylim(-0.5E6, 0.5E6)
#     ax_11[1].scatter(np.arange(50_000), np.array(flt_wf_sum[ch_id]), marker='o', color='green', label = f'og {ch_id}')
#     ax_11[1].scatter(negative_sums, np.array(flt_wf_sum[ch_id])[negative_sums], marker='+', color='blue', label = f'flt {ch_id}')
#     # ax_11[1].legend()
#     ax_11[1].set_xlim((0, 5000))
#     ax_11[1].set_ylim(-5, 5)
# # plt.show()
# fig_11.suptitle('sum flt vs original for negative wf sum events')

In [ ]:
# del Manager, m

In [ ]:
# sys.exit

## 3.1. <a id='toc3_1_'></a>[Event information](#toc0_)

In [ ]:
# # nev_max = m.config('base', 'nevents', 'int')
# # nev_max = 2
# # nev_max = 455 # diag
# # for nev, event in enumerate(m.midas): # oddly doing m.midas shifts the 'pointer' to the next event
# for nev, event in enumerate(m.midas):
#     #print(nev, event, event.nchannels)
#     if nev < 2: continue
#     if nev > 1: break
#     # if nev > 450: break
#     # print(nev)

In [ ]:
# event.adc_baseline

In [ ]:
# # to use this cell, run the above cell once
# plt.close(1)
# plt.figure(1)
# #### Note: this way of looping skips one event every time we execute the for loop. Probably better not to use this loop in full code. 
# for nev, event in enumerate(m.midas): # oddly doing m.midas shifts the 'pointer' to the next event
#     if nev > 0: break
#     wfs = event.adc_data
#     print(event.event_counter)
#     for i,wf in enumerate(event.adc_data): # over channels
#         if event.event_counter:
#             wfs[i] = wf-event.adc_baseline[i]
#             print(event.event_counter)
#             plt.plot(wfs[i], label=f'{i}')
# # plt.xlim(tmin,tmax)
# # plt.legend()

In [ ]:
# del wf
# event.adc_data
# event.adc_baseline
# wf
# wfs[2]
# np.sum(wfs[2][:400])
# np.sum(wf[:400])

In [ ]:
# for nev, event in enumerate(m.midas): # oddly doing m.midas shifts the 'pointer' to the next event
#     if nev > 0: break
#     wfs = event.adc_data
#     if len(event.adc_data) != 0:
#         plt.figure()
#         for i,wf in enumerate(event.adc_data): # over channels
#             wfs[i] = wf-event.adc_baseline[i]
#             plt.plot(wfs[i], label=f'{i}')
#             # plt.xlim(tmin,tmax)
#             plt.legend()

In [ ]:
# event.event_counter
# event.adc_data.shape
# event.adc_data = np.array([])
# len(event.adc_data)
# if not event.adc_data:
#     print('Yes')
# event.adc_baseline
# wfs

In [ ]:
# sys.exit()

In [ ]:
# m.midas
# m.midas.event

In [ ]:
# non_zero_array_cntr = 0
# for arb_iter in range(10000):
#     try:
#         m.midas.read().__getstate__()
#         if m.midas.read().nsamples:
#             non_zero_array_cntr +=1 
#     except IndexError:
#         continue

# print(f'non_zero_array_cntr: {non_zero_array_cntr}')

In [ ]:
# non_zero_array_cntr = 0
# for arb_iter in range(10000):
#     m.midas.read().__getstate__()
#     if m.midas.read().nsamples:
#         non_zero_array_cntr +=1 

# print(f'non_zero_array_cntr: {non_zero_array_cntr}')

In [ ]:
# m.midas.read().__getstate__()
# m.midas.read().zlecompressed # This shows whether data is zle compressed.
# m.midas.read().nsamples
# m.midas.read().nsamples # number of samples
# m.midas.read().adc_data.shape
### m.midas.read().event_size
# m.midas.read().trigger_time
# m.midas.read().n32samples # not an attribute
# m.midas.nsamples not an attribute

# 4. <a id='toc4_'></a>[Analysis](#toc0_)

## 4.1. <a id='toc4_1_'></a>[data selection](#toc0_)

In [ ]:
# event_id = 1000

In [ ]:
# wf = wfs[event_id]

In [ ]:
# # n_channel = 2
# n_channel = 0

In [ ]:
# tmin = 0
# tmax = min(wf[n_channel].shape[0], 4000)

## 4.2. <a id='toc4_2_'></a>[Plotting data](#toc0_)

In [ ]:
# plt.close(2)
# plt.figure(2)
# plt.title('transformed data')
# # from matplotlib.offsetbox import AnchoredText
# for _c in range(3):
#     plt.plot(wf[_c], label=f'channel {_c}')
# # plt.plot(wf, label=f'channel {_c}')

# plt.legend()
# plt.grid()


In [ ]:
def plooter(x, l):
    plt.figure()
    for ch_id in range(3):
        plt.plot(wfs[x][ch_id], alpha = 1/(1+ 0.5*ch_id),label = f'Channel {ch_id}')
    plt.grid()
    plt.legend()
    plt.title(f'pretrigger sum = {np.sum(wfs[x][ch_id][:400])}')
    if l:
        plt.xlim([0, 500])

In [ ]:
# def str_line(x):
#     return m*x + c

In [ ]:
# wf[2]

In [ ]:
# curve_fit(str_line, np.arange[0,400], wf[2][0:400])

In [ ]:
# histogram of pretrigger window

In [ ]:
# wfs.shape

## 4.3. <a id='toc4_3_'></a>[waveform sum](#toc0_)

### 4.3.1. <a id='toc4_3_1_'></a>[filtered waveform sum](#toc0_)

In [ ]:
flt_wf_sum = {
    0: [],
    1: [],
    2: []
}

for event_id in range(wfs.shape[0]):
    flt_wf =  np.sum(perform_arma(wfs[event_id]), axis=1)
    flt_wf_sum[1].append( flt_wf[1] )
    flt_wf_sum[2].append( flt_wf[2] )
    flt_wf_sum[0].append( flt_wf[0] )

In [ ]:
flt_wf_sum[1]

In [ ]:
wf_sum_dict[1][0]

In [ ]:
hist_plot_range = (-100, 275)
fig_3, ax_3 = plt.subplots( 1, 1, figsize=(10, 8), sharex=True, sharey = False)
bin_content_0, bin_edges, _PlotsObjects = ax_3.hist(flt_wf_sum[0], bins=100, range=hist_plot_range, label = 'filtered wf sum 0')

### 4.3.2. <a id='toc4_3_2_'></a>[pretrigger sum](#toc0_)

In [ ]:
ch_x = 1
# wf_sum_ls = []
wf_sum_dict = {
    1: [],
    2: [],
    0: []
}
# pretrigger_sum_ls = []
pretrigger_sum_dict = {
    1: [],
    2: [],
    0: []
}
negative_sums = []
for event_id in range(wfs.shape[0]):
    for ch_id in range(3):
# for event_id in range(1000):
    # pretrigger_sum_ls.append(np.sum(wfs[event_id][ch_x][:400]))
    # pretrigger_sum_ls.append(np.sum(wfs[event_id][ch_x][:350]))
    # wf_sum_ls.append(np.sum(wfs[event_id], axis=1))
        wf_sum_dict[ch_id].append(np.sum(wfs[event_id][ch_id]))
        pretrigger_sum_dict[ch_id].append(np.sum(wfs[event_id][ch_id][:350]))

In [ ]:
# com_negative_sum_ls = []
# sum_negative_sum_ls = []
# for negative_i in negative_sums:
#     com_negative_sum_ls.append(calculate_com(negative_i)[ch_x])
#     sum_negative_sum_ls.append(np.mean(wfs[negative_i][ch_x]))
wf_sum_ls

In [ ]:
pretrigger_sum = {
    0: [],
    1: [],
    2: []
}

for event_x in range(wfs.shape[0]):
    # wf_sum_ls.append(np.sum(wfs[event_x][ch_id])) # channel 0 sum
    pretrigger_sum[1].append( np.sum(wfs[event_x][1][:350]) )
    pretrigger_sum[2].append( np.sum(wfs[event_x][2][:350]) )
    pretrigger_sum[0].append( np.sum(wfs[event_x][0][:350]) )

In [ ]:
fig_0, ax_0 = plt.subplots( 3, 1, figsize=(10, 8), sharex=True, sharey = False)
for ch_x in range(3):
    ax_0[ch_x].hist(pretrigger_sum[ch_x], bins = np.arange(-6000, 200_000, 1000), 
                            color=f'C{ch_x}', label = f'pretrigger sum in channel {ch_x}')
    ax_0[ch_x].set_yscale('log')
    ax_0[ch_x].legend()
    ax_0[ch_x].grid()
    plt.subplots_adjust(wspace=0.025, hspace=0.025)
    fig_0.suptitle('hist of pretrigger sum')
# fig_0.savefig('hist_pretrigger_sum.pdf')
# plt.close(fig_0)

In [ ]:
# sum_arr = np.array(pretrigger_sum_ls)
# sum_arr[np.where(sum_arr <= 20_000)].shape
# np.sort(sum_arr[np.where(sum_arr <= 0)])
# np.sort(sum_arr[np.where(sum_arr <= 3000)])[-100:-10:]
# sum_arr[np.where(sum_arr <= 3000)].shape
# l = np.where(sum_arr <= 25000)
# m = sum_arr[l]
# m
# np.sort(m[np.where(m  >= 3000)])[0]
# np.where(sum_arr <= 6000)
# sum_arr[np.where(sum_arr <= 3000)].shape
# sum_arr[np.where(sum_arr <= 6000)].shape
# np.where(sum_arr == 2913)
# plooter(4699, 0)
# plooter(4699, 1)
# plooter(49971, 0)
# plooter(49971, 1)
# plooter(43686, 0)
# plooter(43686, 1)
# plooter(25129, 0)
# np.sum(np.abs(wfs[25129][2][:350]))
# plooter(25129, 1)

In [ ]:
# plt.figure()
# plt.plot(wfs[25129][0], label='0')
# plt.plot(wfs[25129][1], label='1')
# plt.plot(wfs[25129][2], alpha=0.5, label='2')
# plt.title('Event 25129')
# plt.grid()
# plt.legend()

In [ ]:
# np.sum(wfs[25129][2][:350])
# plooter(34444, 0)

flagged events
1. 34444
2. 25129

In [ ]:
# plt.figure(figsize=(10, 5))
# # plt.plot(wfs[34444][0], label='0')
# # plt.plot(wfs[34444][1], label='1')
# plt.plot(wfs[34444][2], label='2')
# plt.grid()

In [ ]:
# np.sum(wfs[34444][2])

# np.sum(wfs[34444][2][:350])
# plooter(34444, 1)
# plooter(25522, 0)
# plooter(25522, 1)
# plooter(237, 0)
# plooter(237, 1)
# plooter(9370, 0)
# plooter(9370, 1)
# plooter(39661, 0)
# plooter(39661, 1)
# plooter(49316, 0)
# plooter(49316, 1)
# plooter(29020, 0)
# plooter(29020, 1)
# plooter(35670, 0)
# plooter(35670, 1)

### 4.3.3. <a id='toc4_3_3_'></a>[negative wf sum](#toc0_)

merely an investigation

In [ ]:
ch_x = 2

In [ ]:
negative_sums = np.where(np.array(wf_sum_dict[ch_x]) < 0)[0]

In [ ]:
plt.figure()
plt.hist(pretrigger_sum_dict[ch_x], bins = 1000)# bins = np.arange(-6000, 200_000, 1000));
# plt.hist(pretrigger_sum_ls, bins = np.arange(0, 2_000, 100));
plt.title('Histogram pretrigger sum')
plt.yscale('log')

In [ ]:
plt.figure()
plt.plot(wfs[negative_sums[100]][ch_x])

In [ ]:
def calculate_com(event_id):
    event_com = np.divide(np.sum(np.multiply(wfs[event_id], np.arange(wfs[event_id].shape[1])), axis=1), 
                      np.sum(wfs[event_id], axis=1)
                )
    return event_com

In [ ]:
calculate_com(negative_sums[100])

In [ ]:
com_negative_sum_ls = []
sum_negative_sum_ls = []
for negative_i in negative_sums:
    com_negative_sum_ls.append(calculate_com(negative_i)[ch_x])
    sum_negative_sum_ls.append(np.mean(wfs[negative_i][ch_x]))


In [ ]:
plt.figure()
# plt.scatter(negative_sums, sum_negative_sum_ls, label = 'sum')
# plt.scatter(negative_sums, com_negative_sum_ls, label = 'com')
plt.scatter(sum_negative_sum_ls, com_negative_sum_ls, label = 'mean vs com')
plt.legend()

In [ ]:
plt.figure()
# plt.scatter(negative_sums, sum_negative_sum_ls, label = 'sum')
# plt.scatter(negative_sums, com_negative_sum_ls, label = 'com')
plt.scatter(wf_sum_dict[ch_x], com_dict[ch_x], label = 'sum vs com')
plt.xlim(-1e4, 1.5e4)
plt.ylim(-7e3, 1.5e4)
plt.legend()

In [ ]:
mean_com_condition = []
for an_event_id, wf_sum_x in enumerate(wf_sum_dict[ch_x]):
    if wf_sum_x > 0 and com_dict[ch_x][an_event_id] < 0:
        # print(wf_sum_x)
        # print(an_event_id)
        mean_com_condition.append(an_event_id)

In [ ]:
len(mean_com_condition)

In [ ]:
fig_1, ax_1 = plt.subplots(3, 1, figsize=(10,8))
for ch_id in range(3):
    ax_1[ch_id].plot(wfs[mean_com_condition[70]][ch_id], color=f'C{ch_id}')

### 4.3.4. <a id='toc4_3_4_'></a>[Repetitive pattern seen in ch-2 data](#toc0_)

Fourier Transform

In [ ]:
# # event_id = 49998
# event_id = 20_000
# plt.figure()
# for ch in range(3):
#     plt.plot(wfs[event_id][ch], label=f'{ch}')
# plt.legend()
# plt.grid()

#### 4.3.4.1. <a id='toc4_3_4_1_'></a>[Fourier Transform](#toc0_)

In [ ]:
ft_1 = np.fft.fft(wfs[25129][1])
    001, ax_001 = plt.subplots(1, 2, figsize=(10, 5))
ax_001[0].set_title('hist FT of 1')
ax_001[0].hist(ft_1.real, bins=100);
ax_001[1].hist(wfs[25129][1], bins=100);

In [ ]:
ft_2 = np.fft.fft(wfs[25129][2])
fig_002, ax_002 = plt.subplots(1, 2, figsize=(10, 5))
ax_002[0].set_title('hist FT of 2')
ax_002[0].hist(ft_2.real, bins=100);
ax_002[1].hist(wfs[25129][2], bins=100);

## 4.4. <a id='toc4_4_'></a>[simultaneity of pulses](#toc0_)

In [ ]:
def pulse_difference(event_x, use_flt_wf:bool):
    if not use_flt_wf:
        # window_range = np.arange(350, 500)
        window_range = np.arange(350, 4000)
        peaks0 =find_peaks(wfs[event_x][0][window_range])
        peaks1 =find_peaks(wfs[event_x][1][window_range])
        peaks2 =find_peaks(wfs[event_x][2][window_range])
        mp0 = np.argmax(wfs[event_x][0][window_range][peaks0[0]])
        mp1 = np.argmax(wfs[event_x][1][window_range][peaks1[0]])
        mp2 = np.argmax(wfs[event_x][2][window_range][peaks2[0]])
        sample_mp0 = wfs[event_x][0][window_range][peaks0[0]][mp0]
        # sample_mp1 = window_range[peaks1[0]][mp1]
        # sample_mp2 = window_range[peaks2[0]][mp2]

    if use_flt_wf:
        # window_range = np.arange(350, 500)
        window_range = np.arange(350, 4000)
        peaks0 =find_peaks(flt_dict[0][event_x][window_range]) # TODO: better code
        peaks1 =find_peaks(flt_dict[1][event_x][window_range]) # same
        peaks2 =find_peaks(flt_dict[2][event_x][window_range]) # same
        mp0 = np.argmax(flt_dict[0][event_x][window_range][peaks0[0]])
        mp1 = np.argmax(flt_dict[1][event_x][window_range][peaks1[0]])
        mp2 = np.argmax(flt_dict[2][event_x][window_range][peaks2[0]])
        sample_mp0 = flt_dict[0][event_x][window_range][peaks0[0]][mp0]
    
    sample_mp1 = window_range[peaks1[0]][mp1]
    sample_mp2 = window_range[peaks2[0]][mp2]
    # ax_222[1].scatter(window_range[peaks0[0][mp0]], wf_ar[0][window_range][peaks0[0][mp0]], s=2000., marker ='|', color='blue')
    # ax_222[1].scatter(window_range[peaks1[0][mp1]], wf_ar[1][window_range][peaks1[0][mp1]], s=2000., marker ='|', color='orange')
    # ax_222[1].scatter(window_range[peaks2[0][mp2]], wf_ar[2][window_range][peaks2[0][mp2]], s=2000., marker ='|', color='green')

    return abs(sample_mp1 - sample_mp2)

In [ ]:
def create_flt_wfs(wfs):
    flt_dict = {0: [], 
                1: [],
                2: []}
    for og_wf in wfs:
        flt_wf = perform_arma(og_wf)
        for ch_id in range(3):
            flt_dict[ch_id].append(flt_wf[ch_id])
    return flt_dict

flt_dict = create_flt_wfs(wfs)

### 4.4.1. <a id='toc4_4_1_'></a>[Create Pulse difference Distribution](#toc0_)

In [ ]:
pulse_difference_ls = []
for event_id in range(wfs.shape[0]):
    pulse_difference_ls.append( pulse_difference(event_id, use_flt_wf=True) )

In [ ]:
fig_226, ax_226 = plt.subplots(1, figsize = (10, 8))
fig_226.suptitle('Histogram of Pulse differnce between channel 1 and channel 2')
ax_226.hist(pulse_difference_ls, bins=np.arange(0, 2500, 5),
            color=f'C0', label='channels 1 & 2');
ax_226.axvline(x = 40, linestyle='--', color='red')
ax_226.set_yscale('log')
ax_226.set_xlabel('pulse maxima difference in bin units')

In [ ]:
type(fig_226)

In [ ]:
counter_tk = 0

In [ ]:
event_id = event_Fail_array[counter_tk] #66 # 16000
wf_ar = perform_arma( wfs[event_id])
# window_range = np.arange(350, 500)
# window_range = np.arange(350, 4000)
window_range = np.arange(0, 4000)
fig_222, ax_222 = plt.subplots(2, 1, figsize=(10, 8))
ax_222[0].plot(window_range, wfs[event_id][0][window_range], alpha=0.5, label = '0')
ax_222[0].plot(window_range, wfs[event_id][1][window_range], alpha=0.5, label = '1')
ax_222[0].plot(window_range, wfs[event_id][2][window_range], alpha=0.5, label = '2')
ax_222[1].plot(window_range, wf_ar[0][window_range], alpha=0.5, label = '0')
ax_222[1].plot(window_range, wf_ar[1][window_range], alpha=0.5, label = '1')
ax_222[1].plot(window_range, wf_ar[2][window_range], alpha=0.5, label = '2')
ax_222[0].legend()
ax_222[1].legend()
ax_222[0].grid()
ax_222[1].grid()
ax_222[0].set_title('original wf')
ax_222[1].set_title('filtered wf')
# ax_222[0].set_xlim(0, 350)
# ax_222[1].set_xlim(0, 350)
counter_tk += 100
fig_222.suptitle(f'peaks for {event_id}')
print('AR pulse difference:', pulse_difference(event_id, True))

In [ ]:
event_id

In [ ]:
peaks0 =find_peaks(wf_ar[0][window_range])
peaks1 =find_peaks(wf_ar[1][window_range])
peaks2 =find_peaks(wf_ar[2][window_range])

In [ ]:
# ax_222[1].scatter(window_range[peaks0[0]], wf_ar[0][window_range][peaks0[0]], s=None, marker ='o', color='red')
# ax_222[1].scatter(window_range[peaks1[0]], wf_ar[1][window_range][peaks1[0]], s=None, marker ='+', color='purple')
# ax_222[1].scatter(window_range[peaks2[0]], wf_ar[2][window_range][peaks2[0]], s=None, marker ='*', color='magenta')

In [ ]:
mp0 = np.argmax(wf_ar[0][window_range][peaks0[0]])
mp1 = np.argmax(wf_ar[1][window_range][peaks1[0]])
mp2 = np.argmax(wf_ar[2][window_range][peaks2[0]])

In [ ]:
pulse_difference(event_id, False)

In [ ]:
pulse_difference(event_id, True) # ar-161 wf-405 is too much

In [ ]:
np.sum(wfs[event_id][0][:350])

In [ ]:
# pretrigger_sum_post_cut_ls = []
# for event_id in range(wfs.shape[0]):
#     if np.sum(wfs[event_id][ch_x][:350]) <= 20000:
#         if pulse_difference(event_id) <= 40:
#             pretrigger_sum_post_cut_ls.append(np.sum(wfs[event_id][ch_x][:350]))

In [ ]:
# plt.figure()
# plt.hist(pretrigger_sum_post_cut_ls, bins = np.arange(-6000, 20000, 200));
# # plt.hist(pretrigger_sum_ls, bins = np.arange(0, 2_000, 100));
# plt.title('Histogram pretrigger sum post cut')
# plt.yscale('log')

In [ ]:
# len(pretrigger_sum_post_cut_ls)

## 4.5. <a id='toc4_5_'></a>[Center of Mass for waveforms](#toc0_)

In [ ]:
# event_id = 49998
# event_id = 25129 # 500
# wfs[event_id][2]
event_id = 276
# event_id = 278

com_array = np.divide(np.sum(np.multiply(wfs[event_id], np.arange(wfs[event_id].shape[1])), axis=1), 
                      np.sum(wfs[event_id], axis=1)
                    )
plt.figure()
plt.plot(wfs[event_id][0], color='blue', alpha=0.2 , label=f'{0}')
plt.plot(wfs[event_id][1], color='orange', alpha=0.2, label=f'{1}')
plt.plot(wfs[event_id][2], color='green', alpha=0.2 , label=f'{2}')
for ch_id in range(3):
    plt.scatter(com_array[ch_id], np.mean(wfs[event_id], axis=1)[ch_id], marker = '*', label = f'com :{ch_id}')
plt.legend()
plt.grid()
plt.title('Center of Mass approach')

In [ ]:
def calculate_com(event_id):
    event_com = np.divide(np.sum(np.multiply(wfs[event_id], np.arange(wfs[event_id].shape[1])), axis=1), 
                      np.sum(wfs[event_id], axis=1)
                )
    return event_com

In [ ]:
com_dict = {0: [],
            1: [],
            2: []
            }
for event_id in range(wfs.shape[0]):
    com_arr = calculate_com(event_id)
    com_dict[0].append(com_arr[0])
    com_dict[1].append(com_arr[1])
    com_dict[2].append(com_arr[2])
del com_arr

In [ ]:
hist_features = {
    0: [],
    1: [],
    2: [],
}

fig_223, ax_223 = plt.subplots( 3, 1, figsize=(10, 8), sharex=True, sharey = True)
for ch_id in range(3):
    hist_features[ch_id] = ( ax_223[ch_id].hist(com_dict[ch_id], bins=np.arange(-5_000, 5_000, 25), 
                        color=f'C{ch_id}', label = f'{ch_id}')
    )
    ax_223[ch_id].legend()
    ax_223[ch_id].grid()
plt.subplots_adjust(wspace=0.025, hspace=0.025)
fig_223.suptitle('hist of Center Of Mass')

In [ ]:
# above 1000 and greater than 1900
# reject if com in three channels are not concurrent --> change approach

In [ ]:
def red_chisq(f_obs, f_exp, fittedparameters):
    chisqr = np.sum((f_obs - f_exp)**2 / f_exp)
    ndf = f_obs.shape[0]
    print('reducde chisqr:', chisqr/(ndf -fittedparameters.shape[0]), '\n')
    return chisqr/(ndf -fittedparameters.shape[0])

In [ ]:
def fit_com_peak(ch_id):
    
    def f_gauss(x, f_mean, f_sigma, f_k):
        return f_k*(1/(f_sigma*np.sqrt(2*np.pi)))*np.exp(-0.5*((x-f_mean)/f_sigma)**2)
    
    hist_content, hist_edges, histObjects = hist_features[ch_id]
    # x_range = np.arange(235, 270)
    x_range = np.arange(239, 268)

    p0_input = [1270, 208, 1e6]
    # x_range = np.arange(220, 281)

    fitted_parameters, pcov = curve_fit(f_gauss, 
                                # hist_edges[:-1], hist_content,
                                hist_edges[x_range], hist_content[x_range], \
                                p0 = p0_input,
                                # bounds = bounds_input,
                                # sigma = np.sqrt(hist_content[x_range]),
                                # absolute_sigma=True,
                                )

    red_chisqr_value = red_chisq(hist_content[x_range], \
        f_gauss(hist_edges[x_range], *fitted_parameters), fitted_parameters
        )

    ax_223[ch_id].plot(hist_edges[x_range], f_gauss(hist_edges[x_range], *fitted_parameters), label='fit', color='red')
    ax_223[ch_id].legend()


    print(f'red_chisqr_value: {red_chisqr_value}')
    print(f'fitted_parameters for {ch_id}: {fitted_parameters}')
    print('\n')
    return fitted_parameters[0], fitted_parameters[1]

In [ ]:
# com_mean_arr = {
#     0:0,
#     1:1,
#     2:2
# }

# com_std_arr = {
#     0:0,
#     1:1,
#     2:2
# }

com_mean_arr = np.zeros([3,])
com_std_arr = np.zeros([3,])
for ch_id in range(3):
    com_mean_arr[ch_id], com_std_arr[ch_id] = fit_com_peak(ch_id)

In [ ]:
del hist_features

In [ ]:
com_mean_arr

In [ ]:
com_std_arr

## 4.6. <a id='toc4_6_'></a>[full waveform sum](#toc0_)

In [ ]:
wf_sum_ls = []
for event_id in range(wfs.shape[0]):
# for event_id in range(1000):
    wf_sum_ls.append(np.sum(wfs[event_id][ch_x]))

In [ ]:
plt.figure()
plt.hist(wf_sum_ls, bins=500, label = 'Channel 0')
plt.legend()
plt.grid()
plt.title(f'hist of waveform sum')
plt.yscale('log')

In [ ]:
pretrigger_sum = {
    0: [],
    1: [],
    2: []
}
wf_sum_ls = []
com_dict = {0: [],
            1: [],
            2: []
            }

for event_id in range(wfs.shape[0]):
    pretrigger_sum[1].append( np.sum(wfs[event_id][1][:350]) )
    pretrigger_sum[2].append( np.sum(wfs[event_id][2][:350]) )
    pretrigger_sum[0].append( np.sum(wfs[event_id][0][:350]) )
    wf_sum_ls.append( np.sum(wfs[event_id], axis=1) )
    com_arr = calculate_com(event_id)
    com_dict[0].append(com_arr[0])
    com_dict[1].append(com_arr[1])
    com_dict[2].append(com_arr[2])

In [ ]:
del com_arr

## 4.7. <a id='toc4_7_'></a>[Cuts](#toc0_)

In [ ]:
### cuts
### with cut efficieincy

com_threshold = 300
ch_x = 1
com_below_xsigma = com_mean_arr - 2*com_std_arr
com_above_xsigma = com_mean_arr + 2*com_std_arr
wf_sum_post_1_cut_ls = []
wf_sum_post_2_cut_ls = []
wf_sum_post_3_cut_ls = []
com_post_cut_dict = {0: [],
            1: [],
            2: []
            }
event_list_post_cut = []
wf_sum_post_cut_ls = []

for event_id in range(wfs.shape[0]):
    if pretrigger_sum[0][event_id] <= 4000: # 1st cut: pretrigger sum
        wf_sum_post_1_cut_ls.append(wf_sum_ls[event_id][0])
        if pulse_difference(event_id) <= 40: # 2nd cut: simultaneity of pulses
        # if True:
            wf_sum_post_2_cut_ls.append(wf_sum_ls[event_id][0])
            # if (np.abs(com_dict[0][event_id] - com_dict[1][event_id]) <= com_threshold) and (np.abs(com_dict[2][event_id] - com_dict[1][event_id]) <= com_threshold): # 3rd cut: concurrence of COM
            if (com_dict[ch_x][event_id] <= com_above_xsigma)[ch_x] and (com_dict[ch_x][event_id] >= com_below_xsigma[ch_x]): # 3rd cut: distance from mean COM
                wf_sum_post_3_cut_ls.append(wf_sum_ls[event_id][0])
                event_list_post_cut.append(event_id)
                wf_sum_post_cut_ls.append(np.sum(wfs[event_id][ch_x]))
                com_post_cut_dict[0].append(com_dict[0][event_id])
                com_post_cut_dict[1].append(com_dict[1][event_id])
                com_post_cut_dict[2].append(com_dict[2][event_id])

In [ ]:
com_mean_arr - 1*com_std_arr

In [ ]:
com_mean_arr + 1*com_std_arr

In [ ]:
# hist_plot_range = (-6000, 0.5e6)
# hist_plot_range = (1e6, 1e7)
hist_plot_range = (0e6, 7.5e6)

fig_225, ax_225 = plt.subplots( 4, 1, figsize=(10, 8), sharex=True, sharey = False)

bin_content_1, bin_edges, _PlotsObjects = ax_225[0].hist(wf_sum_post_1_cut_ls, bins=500, range = hist_plot_range, label = '1st cut [Channel 0]')
bin_content_2, bin_edges, _PlotsObjects  = ax_225[1].hist(wf_sum_post_2_cut_ls, bins=bin_edges, label = '2nd cut [Channel 0]')
bin_content_3, bin_edges, _PlotsObjects  = ax_225[2].hist(wf_sum_post_3_cut_ls, bins=bin_edges, label = '3rd cut [Channel 0]')

plt.subplots_adjust(wspace=0.025, hspace=0.025)
fig_225.suptitle('successive cuts')
for subplot_id in range(3):
    ax_225[subplot_id].legend()
    ax_225[subplot_id].grid()
eff_2_1 = np.divide(bin_content_2, bin_content_1, out=np.zeros_like(bin_content_2), where=bin_content_1!=0)
ax_225[3].plot(bin_edges[:-1], eff_2_1, alpha=0.5, color='red', label = 'cut_2/cut_1')
eff_3_2 = np.divide(bin_content_3, bin_content_2, out=np.zeros_like(bin_content_3), where=bin_content_2!=0)
ax_225[3].plot(bin_edges[:-1], eff_3_2, alpha=0.5, color='magenta', label = 'cut_3/cut_2')

ax_225[3].legend()
# ax_225[3].set_yscale('log')
# ax_225[3].set_yscale('linear')
ax_225[0].set_ylabel('counts')
ax_225[1].set_ylabel('counts')
ax_225[2].set_ylabel('counts')
ax_225[3].set_ylabel('ratio')
ax_225[3].set_xlabel('wf sum bin')

In [ ]:
# plt.savefig()
# fig_225.savefig('successive_cuts.pdf')
# plt.close(fig_225)

In [ ]:
# del event_list_post_cut, wf_sum_post_1_cut_ls, wf_sum_post_2_cut_ls, wf_sum_post_3_cut_ls

In [ ]:
len(wf_sum_post_1_cut_ls)

In [ ]:
len(wf_sum_post_2_cut_ls)

In [ ]:
len(wf_sum_post_3_cut_ls)

In [ ]:
# ### for preservation
# # wf_sum_post_cut_ls = []
# # event_list_post_cut = []
# # com_threshold = 1500

# com_post_cut_dict = {0: [],
#             1: [],
#             2: []
#             }

# for event_id in range(wfs.shape[0]):
#     # if pretrigger_sum_ls[event_id] <= 4000:
#     if pretrigger_sum_ls[event_id] <= 20000: # 1st cut: pretrigger sum
#         if pulse_difference(event_id) <= 40: # 2nd cut: simultaneity  
#             com_arr = calculate_com(event_id)
#             # if (np.abs(com_arr[0] - com_arr[1]) <= com_threshold) and (np.abs(com_arr[2] - com_arr[1]) <= com_threshold): # 3rd cut: concurrence of COM
#             if (com_dict[ch_x][event_id] <= com_above_xsigma)[ch_x] and (com_dict[ch_x][event_id] >= com_below_xsigma[ch_x]): # 3rd cut: distance from mean COM
#                 event_list_post_cut.append(event_id)
#                 wf_sum_post_cut_ls.append(np.sum(wfs[event_id][ch_x]))
#                 com_post_cut_dict[0].append(com_arr[0])
#                 com_post_cut_dict[1].append(com_arr[1])
#                 com_post_cut_dict[2].append(com_arr[2])

In [ ]:
com_post_cut_dict[0]

In [ ]:
fig_224, ax_224 = plt.subplots( 3, 1, figsize=(10, 8), sharex=True, sharey = True)
for ch_id in range(3):
    ax_224[ch_id].hist(com_post_cut_dict[ch_id], bins=np.arange(-5_000, 5_000, 25), 
                        color=f'C{ch_id}', label = f'{ch_id}')    
    ax_224[ch_id].legend()
    ax_224[ch_id].grid()
plt.subplots_adjust(wspace=0.025, hspace=0.025)
fig_224.suptitle('hist of Center Of Mass post cuts')

In [ ]:
len(com_post_cut_dict[0])

In [ ]:
plt.figure()
plt.hist(wf_sum_post_cut_ls, bins=500, range = (-27e3, 2.75e6), label = 'Channel 0')
plt.legend()
plt.grid()
plt.title(f'hist of waveform sum post 3 cuts')
plt.yscale('log')

In [ ]:
sys.exit()

In [ ]:
# len(wf_sum_post_cut_ls)

In [ ]:
# 500 in event_list_post_cut

In [ ]:
# 34444 in event_list_post_cut

In [ ]:
# 25129 in event_list_post_cut

In [ ]:
# event_list_post_cut[-1::-1]

In [ ]:
50000 in event_list_post_cut

In [ ]:
len(wf_sum_post_cut_ls)

In [ ]:
bad_event_ls = [49971, 43686, 25129, 34444, 25522, 39661, 49316, 29020, 35670, 35670, 49998]

In [ ]:
int_index = 0
for be in bad_event_ls:
    if  be in event_list_post_cut:
        int_index += 1
        print(f'Bad Event Count:{int_index}. Bad event id: {be}')

In [ ]:
perf_counter() - t0

In [ ]:
sys.exit()

# 5. <a id='toc5_'></a>[Exploration](#toc0_)

In [ ]:
wfs.shape

In [ ]:
event_id = 45000

In [ ]:
# for i in range(100_000):
event_id += 1
plooter(event_id, 0)

In [ ]:
event_id

In [ ]:
problem_array = np.load('../scripts/problem_event_169758.npy')
sum_hist_content = np.load('../scripts/wf_sum_0_content.npy')
sum_hist_edges = np.load('../scripts/wf_sum_0_edges.npy')

In [ ]:
### events post selection cuts
event_Pass_array = np.load('../output/event_PassList.npy')
event_Fail_array = np.load('../output/event_FailList.npy')

In [ ]:
event_Pass_array.shape

In [ ]:
event_Fail_array.shape

In [ ]:
problem_array.shape

In [ ]:
plt.figure()
plt.plot(problem_array, label='0')
# plt.xlim(0, 500)
plt.legend()

In [ ]:
np.sum(problem_array[:350])

In [ ]:
plt.figure()
plt.plot(sum_hist_edges[:-1], sum_hist_content)

# 6. <a id='toc6_'></a>[time constant](#toc0_)

In [ ]:
event_Pass_array = np.load('../output/00126_part_output/event_PassList.npy')

In [ ]:
# # summed_flt_wf_dict = {
# stacked_flt_wf_dict = {
# 0:np.zeros_like(wfs[0][0]),
# 1:np.zeros_like(wfs[0][0]),
# 2:np.zeros_like(wfs[0][0]),
# }
# for ch_id in range(3):
#     # summed_flt_wf_dict[ch_id] = np.sum(np.array(flt_dict[ch_id]),axis=0)
#     for event_id in event_Pass_array:
#         stacked_flt_wf_dict[ch_id] += flt_dict[ch_id][event_id]

In [ ]:
# stacked_wf_dict = {
#     0:np.zeros_like(wfs[0][0]),
#     1:np.zeros_like(wfs[0][0]),
#     2:np.zeros_like(wfs[0][0])
# }
# for ch_id in range(3):
#     for event_id in range(wfs.shape[0]):
#         stacked_wf_dict[ch_id] += wfs[event_id][ch_id]

In [ ]:
stacked_flt_wf_dict = pickle.load(open('../output/00126_part_output/stacked_flt_wf_dict.pkl', 'rb'))

In [ ]:
# stacked_wf_dict = pickle.load(open('../output/00126_part_output/stacked_wf_dict.pkl', 'rb'))

In [ ]:
import latexify

In [ ]:
@latexify.function
# def f1_func(x, a0, a1, a2, a3, a4):
#     # x = x + 424
#     a1 = 424
#     return (a0/a2)*np.exp(-(x-a1)/a2) + (a3/a4)*np.exp(-(x-a1)/a4)
def f1_func(x, a0, a2, a3, a4):
    return (a0)*np.exp(-(x)/a2) + (a3)*np.exp(-(x)/a4)

In [ ]:
def red_chisq(f_obs, f_exp, fittedparameters):
    chisqr = np.sum((f_obs - f_exp)**2 / f_exp)
    ndf = f_obs.shape[0]
    print('reducde chisqr:', chisqr/(ndf -fittedparameters.shape[0]), '\n')
    return chisqr/(ndf -fittedparameters.shape[0])

In [ ]:
ch_id = 1
# fit_begin, fit_end = (575, 1600) # 1100
# # fit_begin, fit_end = (800, 2050)
# # f1_range[fit_begin:fit_end]
# bounds_input = ([10.0, 424., 0.0, 0.0, 0.0], [100_000, 425., 1_000.0, 5_000., 10_000.0]) ## ch1
# p0_input = [10000, 3000, 5000, 100]
def perform_fit():
    fitted_parameters, pcov = curve_fit(f1_func,
    # fitted_parameters, pcov = curve_fit(fit_f2, 
                                # f1_range[fit_begin:fit_end], stacked_flt_wf_dict[ch_id][f1_range][fit_begin:fit_end], \
                                f1_range[fit_begin:fit_end], shifted_wf[ch_id][f1_range][fit_begin:fit_end], \
                                p0 = p0_input,
                                # bounds = bounds_input,
                                # sigma = np.sqrt(hist_content[x_range]),
                                # absolute_sigma=True,
                                )
    ax_227[1].plot(f1_range[fit_begin:fit_end], f1_func(f1_range[fit_begin:fit_end], *fitted_parameters), label='fit')
    ax_227[1].legend()
    red_chisq(shifted_wf[ch_id][f1_range][fit_begin:fit_end], 
          f1_func(f1_range[fit_begin:fit_end], *fitted_parameters), 
          fitted_parameters)
    return fitted_parameters
# fitted_parameters = perform_fit()
# ax_227[1].set_yscale('log')

In [ ]:
# p0_input = [10000, 0, 3000, 5000, 100]
# p0_input[1] = 4*np.argmax(stacked_flt_wf_dict[ch_id])
# # # p0_input = [10000,0,3000, 5000, 100]
# # # p0_input = [10000,425,3000]
# # p0_input = [p0/4.0 for p0 in p0_input]
# # p0_input[0] = 250*10**5
# # p0_input[3] = 300*10**3

In [ ]:
# fitted_parameters

In [ ]:
f1_func

In [ ]:
# p0_input

In [ ]:
# bounds_input

In [ ]:
# ax_227[1].plot(f1_range, f1_func(f1_range, *p0_input), label = 'function')
# ax_227[1].legend()

In [ ]:
f1_range = np.arange(0, 3576)

In [ ]:
ch_id = 0
fig_227, ax_227 = plt.subplots(2,1, figsize=(10,8))
for ch_id in range(3):
    ax_227[0].plot(stacked_flt_wf_dict[ch_id], label = f'{ch_id}') 
ax_227[0].legend()
ax_227[0].set_title('stacked wf vs fit of selected events')
ax_227[0].set_yscale('log')
ax_227[0].grid()

shifted_wf = {
    0:0,
    1:1,
    2:2
}
for ch_id in range(3):
    shifted_wf[ch_id] = stacked_flt_wf_dict[ch_id][0+424:4000+424]
    ax_227[1].plot(shifted_wf[ch_id], label = f'{ch_id}')
ax_227[1].set_title('shifted wf')
ax_227[1].legend()
ax_227[1].set_yscale('log')
ax_227[1].grid()

In [ ]:
ch_id = 0
# fit_begin, fit_end = (40, 1250)
# fit_begin, fit_end = (10, 500)
fit_begin, fit_end = (550, 2050)
# fit_begin, fit_end = (500, 1900)# ch1 chisqr=5.56

bounds_input = ([1.0E3, 1E2, 1.0E3, 1E0], [1.0E09, 1.0E6, 1.0E09, 1.0E4]) ## ch1
p0_input = [2500, 4000, 5000, 100]

fitted_parameters, pcov = curve_fit(f1_func,
                            f1_range[fit_begin:fit_end], shifted_wf[ch_id][f1_range][fit_begin:fit_end], \
                            p0 = p0_input,
                            bounds = bounds_input,
                            )
ax_227[1].plot(f1_range[fit_begin:fit_end], f1_func(f1_range[fit_begin:fit_end], *fitted_parameters), label='fit')
ax_227[1].legend()
red_chisq(shifted_wf[ch_id][f1_range][fit_begin:fit_end], 
        f1_func(f1_range[fit_begin:fit_end], *fitted_parameters), 
        fitted_parameters)

In [ ]:
ch_id

In [ ]:
fitted_parameters

In [ ]:
np.argmax(f1_func(f1_range, *p0_input))

In [ ]:
for i in range(3):
    print(np.argmax(stacked_flt_wf_dict[i]))

In [ ]:
ch_id

In [ ]:
###Todo: fit raw waveform

In [ ]:
ch_id =1
ax_227[0].scatter(p0_input[1], stacked_flt_wf_dict[ch_id][int(p0_input[1])], s=200,marker='+', color='magenta')

In [ ]:
p0_input

In [ ]:
sys.exit()

# 7. <a id='toc7_'></a>[Memory diagnostics](#toc0_)

In [ ]:
# ## see memory usage
!cat /proc/meminfo

In [ ]:
# event_catalogue.info(memory_usage='deep')

In [ ]:
# del event_catalogue

In [ ]:
# !cat /proc/meminfo

In [ ]:
# import gc
# gc.isenabled()

In [ ]:
# gc.collect()